In [1]:
 for SQL & Power BI


import pandas as pd
from pathlib import Path

DATA_PATH = Path("/content/data/cleaned_viral_canada_textdates.xlsx")

df = pd.read_excel(DATA_PATH)

print("Dataset Loaded.")
print("Rows:", len(df))
print("Columns:", df.columns.tolist())

df.columns = (
    df.columns.str.lower()
              .str.strip()
              .str.replace(" ", "_")
)

date_cols = ["date", "chart_date", "ref_date"]
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")
        break


numeric_cols = ["rank", "previous_rank", "peak_rank", "days_on_chart"]
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")


df = df.drop_duplicates()

print("Cleaning complete.")


if "date" in df.columns:
    df["year"] = df["date"].dt.year
    df["month"] = df["date"].dt.month
    df["week"] = df["date"].dt.isocalendar().week
    df["day_name"] = df["date"].dt.day_name()

print("Feature engineering done.")



if "artist_names" in df.columns and "days_on_chart" in df.columns:
    artist_longevity = (
        df.groupby("artist_names", as_index=False)["days_on_chart"]
          .sum()
          .sort_values("days_on_chart", ascending=False)
    )

    artist_longevity.to_csv("../data/artist_longevity_summary.csv", index=False)
    print("Saved: artist_longevity_summary.csv")



if "source" in df.columns and "rank" in df.columns:
    label_counts = (
        df.groupby("source", as_index=False)["rank"]
          .count()
          .rename(columns={"rank": "total_chart_entries"})
          .sort_values("total_chart_entries", ascending=False)
    )

    label_counts.to_csv("../data/label_chart_entries_summary.csv", index=False)
    print("Saved: label_chart_entries_summary.csv")



if all(col in df.columns for col in ["track_name", "peak_rank", "previous_rank"]):
    momentum = (
        df.groupby("track_name", as_index=False)[["peak_rank", "previous_rank"]]
          .sum()
          .sort_values("peak_rank", ascending=True)
    )

    momentum.to_csv("../data/rank_momentum_summary.csv", index=False)
    print("Saved: rank_momentum_summary.csv")



print("\nPython Analysis Completed Successfully.")

SyntaxError: cannot assign to expression (ipython-input-3057435759.py, line 1)